In [ ]:
!pip install transformers torch accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.4 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!huggingface-cli whoami

joaosoaresa


In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-13b-hf" # meta-llama/Llama-2-7b-hf

tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## Criando Llama 2 Pipeline


In [ ]:
from transformers import pipeline

llama_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

## Obtendo Respostas

In [ ]:
B_INST, E_INST = "<s>[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

In [ ]:
import pandas as pd
import re

def load_data(file_path: str) -> pd.DataFrame:
    """
    Load data from a CSV file.

    Parameters:
        file_path (str): Path to the CSV file.

    Returns:
        pd.DataFrame: DataFrame containing all data from the CSV file.
    """
    return pd.read_csv(file_path)


In [ ]:
def generate_llama_response(prompt: str) -> str:
    """
    Generate a response from the Llama model.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        str: The model's response.
    """
    sequences = llama_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=4096,
        truncation=True,
        return_full_text=False
    )
    return sequences[0]['generated_text']


In [ ]:
def process_prompt(prompt: dict) -> str:
    """
    Process a prompt and generate a response.

    Parameters:
        prompt (dict): A dictionary containing system and user information.

    Returns:
        str: The generated response.
    """
    user_prompt = f"{B_INST}{B_SYS}{prompt['system']}{E_SYS}{prompt['user']} {E_INST}"
    return generate_llama_response(user_prompt)


In [ ]:
data = pd.read_csv("dataset_final.csv")

def extract_language(path):
    match = re.search(r'\.(\w+)$', path)
    if match:
        return match.group(1).lower()
    else:
        return None

data['language'] = data['path'].apply(extract_language)

In [ ]:
from tqdm import tqdm

def main():
    file_path = "dataset_final.csv"

    df = load_data(file_path)

    instructions = []
    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        language = extract_language(row['path'])
        content = row['content']
        prompt = {
            "system": "If a piece of code doesn't make sense or isn't factually coherent, explain why rather than answering something incorrect. If you don't know the answer to a question, don't share false information.",
            "user": f"Given that you are an experienced programmer, review the code snippet below:\n {row['diff_hunk']} in {language}. \nFormat your review as a json file with concrete instructions to the author of the code.",
            #"user": f"As you are an experienced programmer, please review the code snippet below:\n {row['diff_hunk']} in {language}, taking into account the following context:{content}. \nFormat your review as a JSON file with concrete instructions for the code author."
        }

        response = process_prompt(prompt)
        instructions.append(response)

    # Criar um novo DataFrame com a coluna 'response_llama' adicionada
    df_with_response = df.copy()
    df_with_response['response_llama'] = instructions

    # Salvar o novo DataFrame como CSV mantendo todas as colunas
    df_with_response.to_csv(file_path, index=False)
    print(f"Instructions saved to {file_path}")

if __name__ == "__main__":
    main()


  0%|          | 0/9 [00:00<?, ?it/s]